# Predicting links in Social Networks

In [30]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import *
from sklearn.utils import resample
import pickle

## Load data

### Like relation

In [31]:
like_1 = pd.read_csv("data/engineered_features/Like_Features_day_1.csv")
like_2 = pd.read_csv("data/engineered_features/Like_Features_day_2.csv")
like_3 = pd.read_csv("data/engineered_features/Like_Features_day_3.csv")
like_4 = pd.read_csv("data/engineered_features/Like_Features_day_4.csv")
like_5 = pd.read_csv("data/engineered_features/Like_Features_day_5.csv")

In [32]:
common_1 = pd.read_csv("data/engineered_features/common_interest_day_1.csv")
common_2 = pd.read_csv("data/engineered_features/common_interest_day_2.csv")
common_3 = pd.read_csv("data/engineered_features/common_interest_day_3.csv")
common_3 = pd.read_csv("data/engineered_features/common_interest_day_4.csv")
common_1_3 = pd.read_csv("data/engineered_features/common_interest_combined_day_1_3.csv")

### Group Relation

In [33]:
with open("data/engineered_features/G_features_postings_days_1-3.pkl", "rb") as input_file:
    G_1_3 = pickle.load(input_file)

### Posting TFIDF

In [34]:
with open("data/engineered_features/postings_tfidf_and_graph_features_only_positive_days_1to3.pkl", "rb") as input_file:
    tfidf_pos_1_3 = pickle.load(input_file)

with open("data/engineered_features/postings_tfidf_and_graph_features_only_positive_day_4_test.pkl",
          "rb") as input_file:
    tfidf_pos_4_test = pickle.load(input_file)

with open("data/engineered_features/postings_tfidf_and_graph_features_only_negative_days_1to3.pkl", "rb") as input_file:
    tfidf_neg_1_3 = pickle.load(input_file)

with open("data/engineered_features/postings_tfidf_and_graph_features_only_negative_day_4_test.pkl",
          "rb") as input_file:
    tfidf_neg_4_test = pickle.load(input_file)

## Process data for ML

### Downsample data

In [35]:
TEST_SET_DOWN_FACTOR = 0.2
TRAIN_SET_DOWN_FACTOR = 0.1

Because of the huge number of nodes execution of basic data handling takes long on commodity hardware. Therefore the data sets need to be reduced in size while keeping the characteristics of the orignal data set

#### For the common interest relationship

In [36]:
com = common_1_3
com["label"] = [1 if c > 0 else 0 for c in common_1_3["weight"]]

In [37]:
com_all_downsample = resample(
    com,
    n_samples=int(com.shape[0] * TEST_SET_DOWN_FACTOR)
)

com_neg = com[com.label == 0]
com_pos = com[com.label == 1]

com_neg_downsample = resample(com_neg,
                              n_samples=com_pos.shape[0])

com_resampled = resample(
    pd.concat([com_pos, com_neg_downsample]),
    n_samples=int(com_pos.shape[0]*2*TRAIN_SET_DOWN_FACTOR)
)

In [38]:
com_resampled.shape[0]

20139

The resampled data structure still contains over 200k links, but now the label imbalance is corrected.

#### For the like relationship

In [39]:
like_all = pd.concat([like_1,like_2,like_3,like_4,like_5])
like_all["label"] = [1 if c > 0 else 0 for c in like_all["current_weight"]]

In [40]:
like_all_downsampled = resample(
    like_all,
    n_samples=int(like_all.shape[0] * TEST_SET_DOWN_FACTOR)
)

like_neg = like_all[like_all.label == 0]
like_pos = like_all[like_all.label == 1]

like_neg_downsampled = resample(like_neg,
                              n_samples=like_pos.shape[0])

like_resampled = resample(
    pd.concat([like_pos, like_neg_downsampled]),
    n_samples=int(like_pos.shape[0]*2*TRAIN_SET_DOWN_FACTOR)
)

In [41]:
like_resampled.shape[0]

41171

The resampled data structure still contains over 400k links, but now the label imbalance is corrected.

## Define columns for training

In [42]:
target = "label"
graph_features = [
    "jaccard_coef",
    "adamic_adar_index",
    "preferential_attachment_index",
    "clustering_coefficient_score"
]

X_train_com = com_resampled[graph_features]
y_train_com = com_resampled[target]

X_train_like = like_resampled[graph_features]
y_train_like = like_resampled[target]

In [43]:
rfc_com = RandomForestClassifier(n_estimators=50, max_depth=40, random_state=0)
svm_com = SVC(kernel='linear')
knn_com = KNeighborsClassifier(n_neighbors=10)

rfc_like = RandomForestClassifier(n_estimators=50, max_depth=40, random_state=0)
svm_like = SVC(kernel='linear')
knn_like = KNeighborsClassifier(n_neighbors=10)

## Define evaluation function

In [44]:
def evaluate_model(predictions, actual):
    return pd.DataFrame({
        "Measure": ["Accuracy", "Precision", "Recall", "F1"],
        "Score": [accuracy_score(actual, predictions),
                  precision_score(actual, predictions),
                  recall_score(actual, predictions),
                  f1_score(actual, predictions)]
    })

## Fit classifiers

### RF

In [45]:
rfc_com.fit(X_train_com, y_train_com)

RandomForestClassifier(max_depth=40, n_estimators=50, random_state=0)

In [46]:
rfc_like.fit(X_train_like, y_train_like)

RandomForestClassifier(max_depth=40, n_estimators=50, random_state=0)

### SVM

In [47]:
svm_com.fit(X_train_com, y_train_com)

SVC(kernel='linear')

In [48]:
svm_like.fit(X_train_like, y_train_like)

SVC(kernel='linear')

### KNN

In [49]:
knn_com.fit(X_train_com,y_train_com)

KNeighborsClassifier(n_neighbors=10)

In [50]:
knn_like.fit(X_train_like,y_train_like)

KNeighborsClassifier(n_neighbors=10)

## Evaluate predictions

In [51]:
X_test_com = com_all_downsample[graph_features]
y_test_com = com_all_downsample[target]

In [52]:
X_test_like = like_all_downsampled[graph_features]
y_test_like = like_all_downsampled[target]

### RF evaluation

In [53]:
predictions = rfc_com.predict(X_test_com)
evaluate_model(predictions, y_test_com)

,Measure,Score
0,Accuracy,0.964701
1,Precision,0.123052
2,Recall,0.983904
3,F1,0.218747


In [54]:
predictions = rfc_like.predict(X_test_like)
evaluate_model(predictions, y_test_like)

,Measure,Score
0,Accuracy,0.731963
1,Precision,0.021733
2,Recall,0.700664
3,F1,0.042158


### SVM evaluation

In [55]:
predictions = svm_com.predict(X_test_com)
evaluate_model(predictions, y_test_com)

,Measure,Score
0,Accuracy,0.923128
1,Precision,0.058698
2,Recall,0.951363
3,F1,0.110574


In [56]:
predictions = svm_like.predict(X_test_like)
evaluate_model(predictions, y_test_like)

,Measure,Score
0,Accuracy,0.506062
1,Precision,0.013335
2,Recall,0.790152
3,F1,0.026228


### KNN evaluation

In [57]:
predictions = knn_com.predict(X_test_com)
evaluate_model(predictions, y_test_com)

,Measure,Score
0,Accuracy,0.918471
1,Precision,0.054303
2,Recall,0.927941
3,F1,0.102602


In [58]:
predictions = knn_like.predict(X_test_like)
evaluate_model(predictions, y_test_like)

,Measure,Score
0,Accuracy,0.749327
1,Precision,0.017961
2,Recall,0.536105
3,F1,0.034757


# TODO

- Add engineered features
- Split according to days (one day for test one for verfication?)
- More models
- Grid search for params
- Cross validtation
- Explainability?


See this [repo](https://github.com/neo4j-examples/link-prediction) for reference